In [ ]:
// #r ".\binaries\bossspad.dll"
// #r ".\binaries\XNSEC.dll"

#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
#r "C:\BoSSS\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\XNSEC.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
Init();

## Open Database

In [ ]:
// static var myDb = OpenOrCreateDatabase(@"C:\Databases\BoSSS_DB");
// static var myDb = OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\gutierrez\HeatedCavityNusseltStudy_TRefRepaired"); // Study of nusselt number against changes on dgdegree and cellnumber
static var myDb = OpenOrCreateDatabase(@"D:\DatabasesPaper\HeatedCavity_NusseltStudy"); // 

myDb

{ Session Count = 48; Grid Count = 62; Path = D:\DatabasesPaper\HeatedCavity_NusseltStudy }

In [ ]:
myDb.Sessions

#0: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG3_Ra1000000	09/13/2021 18:48:19	22f319da...
#1: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG4_Ra1000000	09/13/2021 11:02:09	f3643841...
#2: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG2_Ra1000000	09/13/2021 10:27:17	01fe5028...
#3: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG3_Ra1000000	09/13/2021 10:35:56	6162f817...
#4: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG4_Ra1000000	09/13/2021 10:49:58	dbe12dce...
#5: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG1_Ra1000000	09/13/2021 10:16:30	ee94a411...
#6: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG2_Ra1000000	09/13/2021 10:21:38	b6eb2e28...
#7: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG4_Ra10000	09/13/2021 10:08:50	bbb3d34e...
#8: HeatedSquareCavity_Nusselt	NaturalConvection_k4_DG4_Ra1000000	09/13/2021 10:45:15	39a5e587...
#9: HeatedSquareCavity_Nusselt	NaturalConvection_k3_DG4_Ra1000000	09/13/2021 10:45:15	e9be6da8...
#10: HeatedSquareCavit

In [ ]:
BoSSSshell.WorkflowMgm.Init("HeatedSquareCavity_Nusselt");

Project name is set to 'HeatedSquareCavity_Nusselt'.


## Create grid

In [ ]:
int[] Resolutions = new int[]{ 3,4,5,6 };
int[] DGdegree = new int[]{ 1,2,3,4 };
// double[] RaArray = new double []{1e5};

// int[] Resolutions = new int[]{ 6 };
// int[] DGdegree = new int[]{ 4 };
double[] RaArray = new double []{1e2,1e4,1e6};
int[] nCells  =  Resolutions.Select(r => (int)(Math.Pow(2,r+1))).ToArray();



In [ ]:
// int[] Resolutions = new int[]{ /*3,4,5,6*/ 7};
// int[] DGdegree = new int[]{ 1,2,3,4 };
// int[] nCells  =  Resolutions.Select(r => (int)(Math.Pow(2,r+1))).ToArray();
// double[] RaArray = new double []{1e2,1e4,1e6};

In [ ]:
public static class GridFactory {    
 
    public static double[] GetXNodes(int Res) {
 
        var xNodes = GenericBlas.SinLinSpacing(-0.5, 0.5, 0, Res + 1);
        return xNodes;
    }
 
    static double[] GetYNodes(int Res) {
        double[] yNodes =  GenericBlas.SinLinSpacing(-0.5, 0.5,0, Res + 1);
        return yNodes;
    }
 
    public static Grid2D GenerateGrid(int Res) { 
        var xNodes = GetXNodes(Res);
        var yNodes = GetYNodes(Res);
        var grd    = Grid2D.Cartesian2DGrid(xNodes, yNodes);
        grd.EdgeTagNames.Add(1, "NoSlipNeumann");
        grd.EdgeTagNames.Add(2, "wall_tempfixed_left");
        grd.EdgeTagNames.Add(3, "wall_tempfixed_right");
        grd.DefineEdgeTags( delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Adiabatic no slip wall
            //2: Temperature fixed no slip wall

            //right cold wall 
            if (Math.Abs(x - 0.5) < 1e-8)
                return 3;

            //bottom adiabatic Wall
            if (Math.Abs(y - 0.5 ) < 1e-8)
                return 1;

            // left hot wall
            if (Math.Abs(x + 0.5) < 1e-8)
                return 2;

            //top adiabatic Wall
            if (Math.Abs(y + 0.5 ) < 1e-8)
                return 1;

            else throw new ArgumentOutOfRangeException();
        });
        
        bool force = true; 
        myDb.SaveGrid(ref grd, force );
        
        return grd;
     }
 
 }

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double Th, double Tc, double Froude) {
        using(var stw = new System.IO.StringWriter()) {
           
           stw.WriteLine("static class BoundaryValues {");
           stw.WriteLine("  static public double VelX(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double VelY(double[] X) {");
           stw.WriteLine("    return 0.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureHot(double[] X) {");
           stw.WriteLine("    return 1.6;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double TemperatureCold(double[] X) {");
           stw.WriteLine("    return 0.4;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double One(double[] X) {");
           stw.WriteLine("    return  1.0;");
           stw.WriteLine("  }");

           stw.WriteLine("  static public double Zero(double[] X) {");
           stw.WriteLine("    return  0.0;");
           stw.WriteLine("  }");

           stw.WriteLine(" static public double InitialPressure(double[] X) { ");
            stw.WriteLine("         return (-1)* X[1] / ("+Froude * Froude +") ;");
            stw.WriteLine("    }");
            stw.WriteLine("}"); 
           return stw.ToString();
        }
    }
   
    static public Formula Get_VelX(double Th, double Tc , double Froude) {
        return new Formula("BoundaryValues.VelX", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

    static public Formula Get_VelY(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.VelY", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_TemperatureHot(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureHot", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

     static public Formula Get_TemperatureCold(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.TemperatureCold", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }

    static public Formula Get_One(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.One", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_Zero(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.Zero", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
    static public Formula Get_InitialPressure(double Th, double Tc, double Froude){
        return new Formula("BoundaryValues.InitialPressure", AdditionalPrefixCode:GetPrefixCode(Th,Tc,Froude));
    }
}

## Send and run jobs

In [ ]:
var controls = new List<BoSSS.Application.XNSEC.XNSEC_Control>();

In [ ]:
var controls = new List<BoSSS.Application.XNSEC.XNSEC_Control>();
double Th = 1.6; double Tc = 0.4;

foreach(double Ra in RaArray){
foreach(int dg in DGdegree){
foreach(int Res in Resolutions) {

    var C = new BoSSS.Application.XNSEC.XNSEC_Control();
    C.SetDGdegree(dg);
    var nCells = (int) Math.Pow(2,Res+1);
    C.SetGrid(GridFactory.GenerateGrid(nCells));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Res", Res));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Dgdegree", dg));
    C.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("Rayleigh", Ra));

    C.EnableMassFractions = false;
    C.NumberOfChemicalSpecies = 1;
    C.ChemicalReactionActive = false;
    C.MatParamsMode = MaterialParamsMode.Sutherland;
    C.physicsMode = PhysicsMode.Combustion;
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;


    C.LinearSolver = LinearSolverCode.exp_Kcycle_schwarz.GetConfig();
    C.LinearSolver.NoOfMultigridLevels = 5;
    
    // C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;

    C.LinearSolver = LinearSolverCode.direct_pardiso.GetConfig();
    C.NonLinearSolver.ConvergenceCriterion = 1e-11;
    C.LinearSolver.ConvergenceCriterion = 1e-12;
    C.NonLinearSolver.verbose = true;
    C.LinearSolver.verbose = false;
    C.NonLinearSolver.MaxSolverIterations = 200;

    C.PenaltyViscMomentum = 1.0 * 1;
    C.PenaltyHeatConduction = 1.0 * 1;
    C.PhysicalParameters.IncludeConvection = true;

    C.UseSelfMadeTemporalOperator = false;
    C.timeDerivativeEnergyp0_OK = false;
    C.timeDerivativeConti_OK = false;
    bool useHomotopy = false;
    
    if(Ra > 1e4){
        useHomotopy = true;
    }

    if(useHomotopy) { 
        C.HomotopyApproach = BoSSS.Application.XNSEC.XNSEC_Control.HomotopyType.Automatic;        
        C.HomotopyVariable = BoSSS.Application.XNSEC.XNSEC_Control.HomotopyVariableEnum.Reynolds;
        C.homotopieAimedValue = C.Reynolds;
        C.StartingHomotopyValue = Math.Sqrt(1e4);
    }


    C.Rayleigh = Ra;
    C.Reynolds = Math.Sqrt(Ra);
    C.Prandtl = 0.71;
    double Fr =Math.Sqrt(2 * C.Prandtl * (1.6 - 0.4) / (1.6 + 0.4)); 
    C.Froude = Fr;
    C.HeatCapacityRatio = 1.4;
    C.T_ref_Sutherland = 600;
    C.ThermodynamicPressureMode = ThermodynamicPressureMode.MassDetermined; // Because its a closed system, i.e. p0 = p0(time)
    C.PhysicalParameters.IncludeConvection = true;
    C.Timestepper_LevelSetHandling         = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    
    C.SessionName = "NaturalConvection_k" + Res + "_DG" + dg + "_Ra" + Ra; 
  
    
    C.EdgeTagsNusselt = new string[] { "wall_tempfixed_left", "wall_tempfixed_right", "NoSlipNeumann" };

    C.AddBoundaryValue("NoSlipNeumann", VariableNames.VelocityX,BoundaryValueFactory.Get_VelX(Th, Tc,Fr));
    C.AddBoundaryValue("NoSlipNeumann", VariableNames.VelocityY,BoundaryValueFactory.Get_VelY(Th, Tc,Fr));

    C.AddBoundaryValue("wall_tempfixed_left", VariableNames.Temperature,BoundaryValueFactory.Get_TemperatureHot(Th, Tc,Fr));
    C.AddBoundaryValue("wall_tempfixed_right", VariableNames.Temperature, BoundaryValueFactory.Get_TemperatureCold(Th, Tc,Fr));

    C.AddBoundaryValue("wall_tempfixed_left", VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));
    C.AddBoundaryValue("wall_tempfixed_right", VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));





    C.AddInitialValue(VariableNames.VelocityX,  BoundaryValueFactory.Get_Zero(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.VelocityY,BoundaryValueFactory.Get_Zero(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.Pressure,BoundaryValueFactory.Get_InitialPressure(Th, Tc,Fr));

    C.AddInitialValue(VariableNames.Temperature,BoundaryValueFactory.Get_One(Th, Tc,Fr));
    C.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_One(Th, Tc,Fr));

    //C.AddInitialValu(VariableNames.Temperature, X => (Tc - Th) / 1 * X[0] + Th);

    //C.AddInitialValue.Add(VariableNames.Temperature, X => X[0] * X[0] + X[1] * X[1] + 1);
    C.AddInitialValue(VariableNames.ThermodynamicPressure,BoundaryValueFactory.Get_One(Th, Tc,Fr));


    controls.Add(C);
}
}
}

In [ ]:
Console.WriteLine(controls.Count);

## Run Simulations

In [ ]:
BoSSSshell.ExecutionQueues.ForEach(q => Console.WriteLine(q))

In [ ]:
var myBatch = BoSSSshell.ExecutionQueues[3];

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);
foreach(var c in controls) {
    string jobName       = c.SessionName + 1;
    var oneJob           = new Job(jobName, solver);
    int noOfProcs = Convert.ToInt32(c.Paramstudy_CaseIdentification[0].Item2) > 2 ? 4:1;
     noOfProcs = Convert.ToInt32(c.Paramstudy_CaseIdentification[0].Item2) > 4 ? 8:noOfProcs;
    oneJob.NumberOfMPIProcs = noOfProcs;
    oneJob.NumberOfThreads = 1;
    oneJob.SetControlObject(c);
    oneJob.Activate(myBatch); 
}

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

In [ ]:
//myDb.Sessions[2].KeysAndQueries["NusseltNumber0"]

In [ ]:
myDb.Sessions

#0: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG3_Ra1000000	09/13/2021 18:48:19	22f319da...
#1: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG4_Ra1000000	09/13/2021 11:02:09	f3643841...
#2: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG2_Ra1000000	09/13/2021 10:27:17	01fe5028...
#3: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG3_Ra1000000	09/13/2021 10:35:56	6162f817...
#4: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG4_Ra1000000	09/13/2021 10:49:58	dbe12dce...
#5: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG1_Ra1000000	09/13/2021 10:16:30	ee94a411...
#6: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG2_Ra1000000	09/13/2021 10:21:38	b6eb2e28...
#7: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG4_Ra10000	09/13/2021 10:08:50	bbb3d34e...
#8: HeatedSquareCavity_Nusselt	NaturalConvection_k4_DG4_Ra1000000	09/13/2021 10:45:15	39a5e587...
#9: HeatedSquareCavity_Nusselt	NaturalConvection_k3_DG4_Ra1000000	09/13/2021 10:45:15	e9be6da8...
#10: HeatedSquareCavit

## Post Processing: Study of nusselt numbers

In [ ]:
double RaArr = 1e6;
double[] NusseltRefArr = new double[]{0.9787, 2.2180, 8.6870 } ;

In [ ]:
var pDegSessions = myDb.Sessions.Where(
    Si => ((Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == 2) 
     &&(Convert.ToDouble(Si.KeysAndQueries["Rayleigh"])) == 1e6
   ) 
 ).ToArray();


#0: HeatedSquareCavity_Nusselt	NaturalConvection_k6_DG2_Ra1000000	09/13/2021 10:27:17	01fe5028...
#1: HeatedSquareCavity_Nusselt	NaturalConvection_k5_DG2_Ra1000000	09/13/2021 10:21:38	b6eb2e28...
#2: HeatedSquareCavity_Nusselt	NaturalConvection_k4_DG2_Ra1000000	09/13/2021 10:19:39	ec432994...
#3: HeatedSquareCavity_Nusselt	NaturalConvection_k3_DG2_Ra1000000	09/13/2021 10:19:38	10a67177...


In [ ]:
int c = 0;
double TemperatureDifference = 1.6 - 0.4;
foreach(var Ra in RaArray){
    double NussRef = NusseltRefArr[c];
    c++;
    Dictionary<int,double[]> All_NULeftvalues = new Dictionary<int,double[]>();
    Dictionary<int,double[]> All_NURightvalues = new Dictionary<int,double[]>();
    Dictionary<int,double[]> All_NUTopvalues = new Dictionary<int,double[]>();
    Dictionary<int,double[]> All_CellLengths = new Dictionary<int,double[]>();
    Dictionary<int,double[]> All_NusseltsRef = new Dictionary<int,double[]>();

foreach (int pDeg in DGdegree) {
  // int pDeg = 1;
    var pDegSessions = myDb.Sessions.Where(
         Si => ((Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg) 
          &&(Convert.ToDouble(Si.KeysAndQueries["Rayleigh"])) == Ra
        ) 
      ).ToArray();

    double[] leftvalues = new double[pDegSessions.Length];
    double[] rightvalues = new double[pDegSessions.Length];
    double[] residualValueS = new double[pDegSessions.Length];
    double[] hMinArray = new double[pDegSessions.Length];
    double[] RefNusseltArray = new double[pDegSessions.Length];
    
    int i = 0;
    foreach (var sess in pDegSessions) {
        var leftvalue  = sess.KeysAndQueries["NusseltNumber0"];
        var rightvalue = sess.KeysAndQueries["NusseltNumber1"];
        var residualValue = sess.KeysAndQueries["NusseltNumber2"];
        var hmin = sess.KeysAndQueries["Grid:hMin"];
        leftvalues[i] = System.Convert.ToDouble(leftvalue)/TemperatureDifference;
        rightvalues[i] = System.Convert.ToDouble(rightvalue)/TemperatureDifference;
        residualValueS[i] = System.Convert.ToDouble(residualValue)/TemperatureDifference;
        hMinArray[i] = 1.0/System.Convert.ToDouble(hmin);
        RefNusseltArray[i] = NussRef;
        rightvalues[i]*=-1;
        i++;
    }
        All_NULeftvalues.Add(pDeg,leftvalues);
        All_NURightvalues.Add(pDeg,rightvalues);
        All_NUTopvalues.Add(pDeg,residualValueS);
        All_CellLengths.Add(pDeg,hMinArray);
        All_NusseltsRef.Add(pDeg,RefNusseltArray);
}
string basepath = @"C:\tmp\NusseltNumberFinal\";

foreach (int dgDg in DGdegree) {
    string names   = "NusseltReLeft" + Ra.ToString() +"Dg" + dgDg.ToString() + ".txt";
    string txtpath = basepath + names;
    using (StreamWriter stw = new StreamWriter(txtpath)) {
        stw.WriteLine("x\ty");
        for (int i = 0; i < 4; i++) {
            stw.Write(All_CellLengths[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            stw.Write(All_NULeftvalues[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
            stw.WriteLine();
        }
    }
}

foreach (int dgDg in DGdegree) {
    string names   = "NusseltReRight" + Ra.ToString() +"Dg" + dgDg.ToString() + ".txt";
    string txtpath = basepath + names;
    using (StreamWriter stw = new StreamWriter(txtpath)) {
        stw.WriteLine("x\ty");
        for (int i = 0; i < 4; i++) {
            stw.Write(All_CellLengths[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            stw.Write(All_NURightvalues[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
            stw.WriteLine();
        }
    }
}


foreach (int dgDg in DGdegree) {
    string names   = "ReferenceNusselt" + Ra.ToString() +"Dg" + dgDg.ToString() + ".txt";
    string txtpath = basepath + names;
    using (StreamWriter stw = new StreamWriter(txtpath)) {
        for (int i = 0; i < 4; i++) {
            stw.Write(All_CellLengths[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            stw.Write(All_NusseltsRef[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
            stw.WriteLine();
            Console.WriteLine(All_CellLengths[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
        }
    
    }
}



}

1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	
1.2800000000000000E+002	
6.4000000000000000E+001	
3.2000000000000000E+001	
1.6000000000000000E+001	


## Calculation of p0 and Nusselt numbers on a fine mesh
used for comparison with reference values

In [ ]:
double[] p0s = new double[6];
double[] NuLeft = new double[6];
double[] NuRight = new double[6];
for(int i = 0; i < p0s.Length; i++){

    NuLeft[i] = Convert.ToDouble(myDb.Sessions[i].KeysAndQueries["NusseltNumber0"])/1.2;
    NuRight[i] = Convert.ToDouble(myDb.Sessions[i].KeysAndQueries["NusseltNumber1"])/1.2;
}
for(int i = 0; i < p0s.Length; i++){
    Console.WriteLine(i);
    p0s[i] = myDb.Sessions[i].Timesteps.Last().Fields.Where(f=>f.Identification == "ThermodynamicPressure").First().GetMeanValue(0);

}